In [1]:
from openai import OpenAI
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ems = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

In [3]:
def emotions_to_categorical(df):
    res = []

    for i in df['emotions']:
        tmp = [0 for _ in range(28)]
        for j in i:
            tmp[j] = 1
        res.append(tmp)
    tmp_df = pd.DataFrame(res, columns=ems)
    
    return tmp_df

In [4]:
def emotions_to_ekman(df):
    # anger disgust fear joy sadness surprise neutral
    ekman = [3, 3, 0, 0, 3, 3, 5, 5, 3, 4, 0, 1, 4, 3, 2, 3, 4, 3, 3, 2, 3, 3, 5, 3, 4, 4, 5, 6]
    res = []

    for i in df:
        tmp = [0, 0, 0, 0, 0, 0, 0]
        for j in range(len(i)):
            if i[j] == 1:
                tmp[ekman[j]] = 1
        res.append(tmp)
    tmp_df = pd.DataFrame(res, columns=['angry', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'])
    
    return tmp_df

In [5]:
def data_init(path = "../../data/dev.tsv"):
    df = pd.read_csv(path, sep="\t", encoding = "utf-8", header=None)
    df.columns = ['text', 'emotions', 'id']
    df['emotions'] = list(map(lambda s : list(map(int, s.split(','))), df['emotions']))
    df = pd.concat([df, emotions_to_categorical(df)], axis=1)
    df = df.drop(columns=['emotions'])
    df['text'] = list(map(lambda s : s.replace('\\', '\\\\').replace('"', '\\"'), list(df['text']))) 
    return df.iloc[:2500, :]

In [6]:
def evaluation(original_df, emotion_res):
    emotions_list = ems
    df = original_df
    predicted_df = pd.DataFrame(data = [[0 for _ in range(28)] for _ in range(len(df))], columns=emotions_list)
    for i in range(len(emotion_res)):
        for j in emotion_res[i]:
            if j in emotions_list:
                predicted_df.loc[i, j] = 1
    predicted = predicted_df.to_numpy()
    original = df.to_numpy()
    
    
    accuracy = accuracy_score(original, predicted)
    
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        original, predicted, average='micro'
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        original, predicted, average='macro'
    )
    
    precision_per_label, recall_per_label, f1_per_label, _ = precision_recall_fscore_support(
        original, predicted, average=None
    )

    precision_macro_std = np.std(precision_per_label)
    recall_macro_std = np.std(recall_per_label)
    f1_macro_std = np.std(f1_per_label)

    print("--- 모델 평가 결과 ---")
    print(f"전체 샘플에 대한 정확도 (Exact Match Accuracy): {accuracy:.4f}")
    print("\n--- Micro 평균 지표 ---")
    print(f"Precision (Micro): {precision_micro:.4f}")
    print(f"Recall (Micro): {recall_micro:.4f}")
    print(f"F1-Score (Micro): {f1_micro:.4f}")
    print("\n--- Macro 평균 지표 ---")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    
    print("\n--- 라벨별 지표 ---")
    for i in range(len(emotions_list)):
        print(f"{emotions_list[i]} - Precision: {precision_per_label[i]:.4f}, Recall: {recall_per_label[i]:.4f}, F1-Score: {f1_per_label[i]:.4f}")
    
    print(f"\nPrecision (Macro) 표준편차: {precision_macro_std:.4f}")
    print(f"Recall (Macro) 표준편차: {recall_macro_std:.4f}")
    print(f"F1-Score (Macro) 표준편차: {f1_macro_std:.4f}")

    return accuracy, f1_micro, f1_macro, precision_recall_fscore_support(original, predicted, average='macro')

In [7]:
def evaluation_ekman(original_df, emotion_res):
    emotions_list = 'anger disgust fear joy sadness surprise neutral'.split()
    predicted_df = pd.DataFrame(data = [[0 for _ in range(28)] for _ in range(len(original_df))], columns=ems)
    for i in range(len(emotion_res)):
        for j in emotion_res[i]:
            if j in ems:
                predicted_df.loc[i, j] = 1
    predicted = emotions_to_ekman(predicted_df.to_numpy()).to_numpy()
    original = emotions_to_ekman(original_df.to_numpy()).to_numpy()

    accuracy = accuracy_score(original, predicted)
    
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        original, predicted, average='micro'
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        original, predicted, average='macro'
    )
    
    precision_per_label, recall_per_label, f1_per_label, _ = precision_recall_fscore_support(
        original, predicted, average=None
    )

    precision_macro_std = np.std(precision_per_label)
    recall_macro_std = np.std(recall_per_label)
    f1_macro_std = np.std(f1_per_label)

    print("--- 모델 평가 결과 ---")
    print(f"전체 샘플에 대한 정확도 (Exact Match Accuracy): {accuracy:.4f}")
    print("\n--- Micro 평균 지표 ---")
    print(f"Precision (Micro): {precision_micro:.4f}")
    print(f"Recall (Micro): {recall_micro:.4f}")
    print(f"F1-Score (Micro): {f1_micro:.4f}")
    print("\n--- Macro 평균 지표 ---")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    
    print("\n--- 라벨별 지표 ---")
    for i in range(len(emotions_list)):
        print(f"{emotions_list[i]} - Precision: {precision_per_label[i]:.4f}, Recall: {recall_per_label[i]:.4f}, F1-Score: {f1_per_label[i]:.4f}")
    
    print(f"\nPrecision (Macro) 표준편차: {precision_macro_std:.4f}")
    print(f"Recall (Macro) 표준편차: {recall_macro_std:.4f}")
    print(f"F1-Score (Macro) 표준편차: {f1_macro_std:.4f}")

    return accuracy, f1_micro, f1_macro, precision_recall_fscore_support(original, predicted, average='macro')

In [8]:
def file_init():
    file_dict = {}
    basicPath = '../prompt/multi_v2'
    file_names = {
        'neutral_persona': basicPath+'/neutral/persona.txt',
        'neutral_guidelines': basicPath+'/neutral/guidelines.txt',
        'positive_persona': basicPath+'/positive/persona.txt',
        'positive_guidelines': basicPath+'/positive/guidelines.txt',
        'positive_description': basicPath+'/positive/emotion_description.txt',
        'ambiguous_persona': basicPath+'/ambiguous/persona.txt',
        'ambiguous_guidelines': basicPath+'/ambiguous/guidelines.txt',
        'ambiguous_description': basicPath+'/ambiguous/emotion_description.txt',
        'disgust_persona': basicPath+'/disgust/persona.txt',
        'disgust_guidelines': basicPath+'/disgust/guidelines.txt',
        'disgust_description': basicPath+'/disgust/emotion_description.txt',
        'anger_persona': basicPath+'/anger/persona.txt',
        'anger_guidelines': basicPath+'/anger/guidelines.txt',
        'anger_description': basicPath+'/anger/emotion_description.txt',
        'fear_persona': basicPath+'/fear/persona.txt',
        'fear_guidelines': basicPath+'/fear/guidelines.txt',
        'fear_description': basicPath+'/fear/emotion_description.txt',
        'sadness_persona': basicPath+'/sadness/persona.txt',
        'sadness_guidelines': basicPath+'/sadness/guidelines.txt',
        'sadness_description': basicPath+'/sadness/emotion_description.txt'
    }
    for key, value in file_names.items():
        file = open(value, 'r')
        file_dict[key] = file.read()
        file.close()
    return file_dict

In [9]:
data = data_init()

In [10]:
key_file = open('../key/openai_key.txt', 'r')
api_key = key_file.readline()
key_file.close()
client = OpenAI(api_key=api_key)

In [11]:
class Agent:
    def __init__(self, system, format, filename, outputCol, input, inputCol):
        self.query = {
        "custom_id": "",
        "method": "POST",
        "url": "/v1/responses",
        "body": {
                "model": "gpt-4o-mini",
                "temperature": 0.5,
                "top_p": 1.0,
                "input": [{
                    "role": "developer",
                    "content": f"{system}"
                }, 
                {
                    "role": "user",
                    "content": ""
                }], 
                "max_output_tokens": 1000,
                "text": {
                    "format": format
                }
            }
        }
        self.filename = filename
        self.batch_id = None
        self.batch_res = None
        self.emotion_res = []
        self.outputCol = outputCol
        self.inputCol = inputCol
        self.input = input
        self.output = None
    
    def make_jsonl(self):
        tmp_query = self.query
        with open(self.filename, "w") as f:
            k = 0
            for i in range(len(self.input)):
                record = {}
                for j in self.inputCol:
                    record[j] = self.input.loc[i, j]
                tmp_query['custom_id'] = f"query{k}"
                tmp_query['body']['input'][1]['content'] = json.dumps(record)
                k += 1
                f.write(json.dumps(tmp_query) + "\n")
            
    def run(self):
        batch_input_file = client.files.create(
            file=open(self.filename, "rb"),
            purpose='batch'
        )
        print(batch_input_file)

        batch_input_file_id = batch_input_file.id
        create_batch=client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/responses",
            completion_window="24h",
        )
        self.batch_id = create_batch.id

    def check_status(self):
        batch = client.batches.retrieve(self.batch_id)
        result = None
        if batch.status == 'completed':
            out = batch.output_file_id
            if out != None:
                print('done!')
                result = client.files.content(out)
                self.batch_res = result
                return 1
            else:
                print('error')
                result = client.files.content(batch.error_file_id).text
                self.batch_res = result
                return -2
        elif batch.status == 'failed':
            print('failed')
            print(batch.errors)
            return -1
        else:
            print('it does not finish yet')
            print(batch.status)
            print(batch.request_counts)
            return 0
    
    def parse_result(self):
        status = self.check_status()
        if status != 1:
            print('can not parse result')
            return
        json_res = []
        for i in self.batch_res.text.split('\n')[:-1]:
            json_res.append(json.loads(i))

        for i in json_res:
            tmp = [[] for _ in range(len(self.outputCol))]
            n = json.loads(i['response']['body']['output'][0]['content'][0]['text'])

            for j in n['analysis']:
                for k in range(len(self.outputCol)):
                    tmp[k].append(j[self.outputCol[k]])

            self.emotion_res.append(tmp)
        tmp_df = pd.DataFrame(data=self.emotion_res, columns=self.outputCol)
        self.output = pd.concat([self.input, tmp_df], axis=1)


In [12]:
multi_positive_format = {
        "type": "json_schema",
        "name": "result",
        "strict": True,
        "schema": {
          "type": "object",
          "properties": {
            "analysis": {
              "type": "object",
              "properties": {
                "admiration": {
                  "type": "integer",
                  "description": "A score on which the text represents a admiration.",
                  "maximum": 100,
                  "minimum": 1
                },
                "amusement": {
                  "type": "integer",
                  "description": "A score on which the text represents a amusement.",
                  "maximum": 100,
                  "minimum": 1
                },
                "approval": {
                  "type": "integer",
                  "description": "A score on which the text represents a approval.",
                  "maximum": 100,
                  "minimum": 1
                },
                "caring": {
                  "type": "integer",
                  "description": "A score on which the text represents a caring.",
                  "maximum": 100,
                  "minimum": 1
                },
                "desire": {
                  "type": "integer",
                  "description": "A score on which the text represents a desire.",
                  "maximum": 100,
                  "minimum": 1
                },
                "excitement": {
                  "type": "integer",
                  "description": "A score on which the text represents a excitement.",
                  "maximum": 100,
                  "minimum": 1
                },
                "gratitude": {
                  "type": "integer",
                  "description": "A score on which the text represents a gratitude.",
                  "maximum": 100,
                  "minimum": 1
                },
                "joy": {
                  "type": "integer",
                  "description": "A score on which the text represents a joy.",
                  "maximum": 100,
                  "minimum": 1
                },
                "love": {
                  "type": "integer",
                  "description": "A score on which the text represents a love.",
                  "maximum": 100,
                  "minimum": 1
                },
                "optimism": {
                  "type": "integer",
                  "description": "A score on which the text represents a optimism.",
                  "maximum": 100,
                  "minimum": 1
                },
                "pride": {
                  "type": "integer",
                  "maximum": 100,
                  "minimum": 1
                },
                "relief": {
                  "type": "integer",
                  "maximum": 100,
                  "minimum": 1
                }
              },
              "required": [
                "admiration",
                "amusement",
                "approval",
                "caring",
                "desire",
                "excitement",
                "gratitude",
                "joy",
                "love",
                "optimism",
                "pride",
                "relief"
              ],
              "additionalProperties": False
            }
          },
          "required": [
            "analysis"
          ],
          "additionalProperties": False
        }
      }

In [13]:
files = file_init()
positive_system = f"{files['positive_persona']}{files['positive_description']}{files['positive_guidelines']}"
positive = Agent(positive_system, multi_positive_format, "../inputs/multi/v2/positive.jsonl", [
                "admiration",
                "amusement",
                "approval",
                "caring",
                "desire",
                "excitement",
                "gratitude",
                "joy",
                "love",
                "optimism",
                "pride",
                "relief"
              ], data, ['text'])

In [14]:
positive.make_jsonl()
positive.run()

FileObject(id='file-JsnssG98kCKWQq7wSYFfkN', bytes=10284108, created_at=1764223450, filename='positive.jsonl', object='file', purpose='batch', status='processed', expires_at=1766815450, status_details=None)


In [ ]:
# positive.batch_id = 'batch_69255cb1da448190902e8aaf10d53b55'
print(positive.batch_id)

In [15]:
multi_ambiguous_format = {
        "type": "json_schema",
        "name": "result",
        "strict": True,
        "schema": {
          "type": "object",
          "properties": {
            "analysis": {
              "type": "object",
              "properties": {
                "realization": {
                  "type": "integer",
                  "description": "A score on which the text represents a realization.",
                  "maximum": 100,
                  "minimum": 1
                },
                "surprise": {
                  "type": "integer",
                  "description": "A score on which the text represents a surprise.",
                  "maximum": 100,
                  "minimum": 1
                },
                "confusion": {
                  "type": "integer",
                  "description": "A score on which the text represents a confusion.",
                  "maximum": 100,
                  "minimum": 1
                },
                "curiosity": {
                  "type": "integer",
                  "description": "A score on which the text represents a curiosity.",
                  "maximum": 100,
                  "minimum": 1
                }
              },
              "required": [
                "realization",
                "surprise",
                "confusion",
                "curiosity"
              ],
              "additionalProperties": False
            }
          },
          "required": [
            "analysis"
          ],
          "additionalProperties": False
        }
      }

In [17]:
files = file_init()
ambiguous_system = f"{files['ambiguous_persona']}{files['ambiguous_description']}{files['ambiguous_guidelines']}"
ambiguous = Agent(ambiguous_system, multi_ambiguous_format, "../inputs/multi/v2/ambiguous.jsonl", ["realization", "surprise", "confusion", "curiosity"], data, ['text'])

In [18]:
ambiguous.make_jsonl()
ambiguous.run()

FileObject(id='file-MjYyAviphTj4CeFRtrBdbY', bytes=6254108, created_at=1764224059, filename='ambiguous.jsonl', object='file', purpose='batch', status='processed', expires_at=1766816059, status_details=None)


In [19]:
# ambiguous.batch_id = 'batch_69255cb1da448190902e8aaf10d53b55'
print(ambiguous.batch_id)

batch_6927ec3c9ac48190b65103e085fdf218


In [38]:
multi_neutral_format = {
    "type": "json_schema",
    "name": "result",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "analysis": {
              "type": "object",
              "properties": {
                "neutral": {
                  "type": "integer",
                  "description": "A score on which the text represents little emotion.",
                  "maximum": 100,
                  "minimum": 1
                }
              },
              "required": [
                "neutral"
              ],
              "additionalProperties": False
            }
        },
        "required": ["analysis"],
        "additionalProperties": False
    }
}

In [39]:
files = file_init()
neutral_system = f"{files['neutral_persona']}{files['neutral_guidelines']}"
neutral = Agent(neutral_system, multi_neutral_format, "../inputs/multi/v2/neutral.jsonl", ['neutral'], data, ['text'])

In [40]:
neutral.make_jsonl()
neutral.run()

FileObject(id='file-R6K8Z6dBj7cqGEmHtT3YtJ', bytes=4211608, created_at=1764224933, filename='neutral.jsonl', object='file', purpose='batch', status='processed', expires_at=1766816933, status_details=None)


In [41]:
# neutral.batch_id = 'batch_69255cb1da448190902e8aaf10d53b55'
print(neutral.batch_id)

batch_6927efa5a75c81908ca797d3b3bca5f7


In [ ]:
neutral.check_status()

done!


1

In [33]:
multi_disgust_format = {
        "type": "json_schema",
        "name": "result",
        "strict": True,
        "schema": {
          "type": "object",
          "properties": {
            "analysis": {
              "type": "object",
              "properties": {
                "disgust": {
                  "type": "integer",
                  "description": "A score on which the text represents a disgust.",
                  "maximum": 100,
                  "minimum": 1
                }
              },
              "required": [
                "disgust"
              ],
              "additionalProperties": False
            }
          },
          "required": [
            "analysis"
          ],
          "additionalProperties": False
        }
      }

In [35]:
files = file_init()
disgust_system = f"{files['disgust_persona']}{files['disgust_description']}{files['disgust_guidelines']}"
disgust = Agent(disgust_system, multi_disgust_format, "../inputs/multi/v2/disgust.jsonl", ["disgust"], data, ['text'])

In [36]:
disgust.make_jsonl()
disgust.run()

FileObject(id='file-NSn5oYnqAh9sSgkJrp59nW', bytes=6039108, created_at=1764224600, filename='disgust.jsonl', object='file', purpose='batch', status='processed', expires_at=1766816600, status_details=None)


In [37]:
# disgust.batch_id = 'batch_69255cb1da448190902e8aaf10d53b55'
print(disgust.batch_id)

batch_6927ee59806c8190af075921f204977d


In [20]:
multi_anger_format = {
        "type": "json_schema",
        "name": "result",
        "strict": True,
        "schema": {
          "type": "object",
          "properties": {
            "analysis": {
              "type": "object",
              "properties": {
                "anger": {
                  "type": "integer",
                  "description": "A score on which the text represents a anger.",
                  "maximum": 100,
                  "minimum": 1
                },
                "annoyance": {
                  "type": "integer",
                  "description": "A score on which the text represents a annoyance.",
                  "maximum": 100,
                  "minimum": 1
                },
                "disapproval": {
                  "type": "integer",
                  "description": "A score on which the text represents a disapproval.",
                  "maximum": 100,
                  "minimum": 1
                }
              },
              "required": [
                "anger",
                "annoyance",
                "disapproval"
              ],
              "additionalProperties": False
            }
          },
          "required": [
            "analysis"
          ],
          "additionalProperties": False
        }
      }

In [21]:
files = file_init()
anger_system = f"{files['anger_persona']}{files['anger_description']}{files['anger_guidelines']}"
anger = Agent(anger_system, multi_anger_format, "../inputs/multi/v2/anger.jsonl", ["anger", "annoyance", "disapproval"], data, ['text'])

In [22]:
anger.make_jsonl()
anger.run()

FileObject(id='file-Xdwh56sMnT5hjkWVmw9aFw', bytes=6901608, created_at=1764224090, filename='anger.jsonl', object='file', purpose='batch', status='processed', expires_at=1766816090, status_details=None)


In [23]:
# anger.batch_id = 'batch_69255cb1da448190902e8aaf10d53b55'
print(anger.batch_id)

batch_6927ec5b313881908894e69f4faa95e6


In [91]:
anger.check_status()

done!


1

In [24]:
multi_fear_format = {
        "type": "json_schema",
        "name": "result",
        "strict": True,
        "schema": {
          "type": "object",
          "properties": {
            "analysis": {
              "type": "object",
              "properties": {
                "fear": {
                  "type": "integer",
                  "description": "A score on which the text represents a fear.",
                  "maximum": 100,
                  "minimum": 1
                },
                "nervousness": {
                  "type": "integer",
                  "description": "A score on which the text represents a nervousness.",
                  "maximum": 100,
                  "minimum": 1
                }
              },
              "required": [
                "fear",
                "nervousness"
              ],
              "additionalProperties": False
            }
          },
          "required": [
            "analysis"
          ],
          "additionalProperties": False
        }
      }

In [25]:
files = file_init()
fear_system = f"{files['fear_persona']}{files['fear_description']}{files['fear_guidelines']}"
fear = Agent(fear_system, multi_fear_format, "../inputs/multi/v2/fear.jsonl", ["fear", "nervousness"], data, ['text'])

In [26]:
fear.make_jsonl()
fear.run()

FileObject(id='file-96GK2j3exWh3DGrm11a7XJ', bytes=6496608, created_at=1764224108, filename='fear.jsonl', object='file', purpose='batch', status='processed', expires_at=1766816108, status_details=None)


In [27]:
# fear.batch_id = 'batch_69255cb1da448190902e8aaf10d53b55'
print(fear.batch_id)

batch_6927ec6d4948819080a5efe9188a5385


In [93]:
fear.check_status()

done!


1

In [28]:
multi_sadness_format = {
        "type": "json_schema",
        "name": "result",
        "strict": True,
        "schema": {
          "type": "object",
          "properties": {
            "analysis": {
              "type": "object",
              "properties": {
                "disappointment": {
                  "type": "integer",
                  "description": "A score on which the text represents a disappointment.",
                  "maximum": 100,
                  "minimum": 1
                },
                "embarrassment": {
                  "type": "integer",
                  "description": "A score on which the text represents a embarrassment.",
                  "maximum": 100,
                  "minimum": 1
                },
                "grief": {
                  "type": "integer",
                  "description": "A score on which the text represents a grief.",
                  "maximum": 100,
                  "minimum": 1
                },
                "remorse": {
                  "type": "integer",
                  "description": "A score on which the text represents a remorse.",
                  "maximum": 100,
                  "minimum": 1
                },
                "sadness": {
                  "type": "integer",
                  "description": "A score on which the text represents a sadness.",
                  "maximum": 100,
                  "minimum": 1
                }
              },
              "required": [
                "disappointment",
                "embarrassment",
                "grief",
                "remorse",
                "sadness"
              ],
              "additionalProperties": False
            }
          },
          "required": [
            "analysis"
          ],
          "additionalProperties": False
        }
      }

In [30]:
files = file_init()
sadness_system = f"{files['sadness_persona']}{files['sadness_description']}{files['sadness_guidelines']}"
sadness = Agent(sadness_system, multi_sadness_format, "../inputs/multi/v2/sadness.jsonl", ["disappointment", "embarrassment", "grief", "remorse", "sadness"], data, ['text'])

In [31]:
sadness.make_jsonl()
sadness.run()

FileObject(id='file-TyCVQq1YzRKxpcNDv48y87', bytes=7636608, created_at=1764224129, filename='sadness.jsonl', object='file', purpose='batch', status='processed', expires_at=1766816129, status_details=None)


In [32]:
# sadness.batch_id = 'batch_69255cb1da448190902e8aaf10d53b55'
print(sadness.batch_id)

batch_6927ec817e1c8190a23cdfc251a57af0


In [95]:
sadness.check_status()

done!


1

In [43]:
neutral.check_status()
positive.check_status()
disgust.check_status()
ambiguous.check_status()
anger.check_status()
fear.check_status()
sadness.check_status()

done!
done!
done!
done!
done!
done!
done!


1

In [ ]:
li = []
for case in [neutral, positive, disgust, ambiguous, anger, fear, sadness]:
    

In [74]:
json_res = []
for i in sadness.batch_res.text.split('\n')[:-1]:
    json_res.append(json.loads(i))
tmp = []
for i in range(len(json_res)):
    for j in [
                "disappointment",
                "embarrassment",
                "grief",
                "remorse",
                "sadness"]:
        scores.loc[i,j] = json.loads(json_res[i]['response']['body']['output'][0]['content'][0]['text'])['analysis'][j]


In [57]:
scores

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral


In [89]:
res = []
for i in range(2500):
    tmp = []
    for j in ems:
        if scores.loc[i, j] > 50:
            tmp.append(j)
    res.append(tmp)

In [90]:
res

[['curiosity', 'neutral'],
 ['anger', 'annoyance', 'disapproval', 'realization'],
 ['sadness'],
 ['curiosity', 'realization', 'neutral'],
 ['neutral'],
 ['disappointment', 'gratitude', 'realization', 'neutral'],
 ['confusion', 'surprise'],
 ['realization', 'surprise', 'neutral'],
 ['caring', 'optimism', 'realization', 'neutral'],
 ['anger', 'annoyance', 'disapproval', 'disgust'],
 ['admiration',
  'approval',
  'excitement',
  'joy',
  'love',
  'optimism',
  'realization',
  'neutral'],
 ['disappointment', 'embarrassment', 'realization', 'sadness', 'neutral'],
 ['grief', 'realization', 'sadness', 'neutral'],
 ['admiration', 'approval', 'confusion', 'excitement', 'pride', 'neutral'],
 ['annoyance', 'disapproval', 'disgust'],
 ['amusement', 'anger', 'annoyance', 'disapproval', 'surprise'],
 ['amusement', 'disappointment', 'excitement'],
 ['neutral'],
 ['admiration', 'approval', 'joy', 'love', 'pride'],
 ['disapproval', 'realization', 'neutral'],
 ['pride', 'neutral'],
 ['admiration', 'c

In [91]:
e = evaluation_ekman(data.iloc[:, 2:30], res)

--- 모델 평가 결과 ---
전체 샘플에 대한 정확도 (Exact Match Accuracy): 0.1528

--- Micro 평균 지표 ---
Precision (Micro): 0.3692
Recall (Micro): 0.6693
F1-Score (Micro): 0.4759

--- Macro 평균 지표 ---
Precision (Macro): 0.3311
Recall (Macro): 0.6388
F1-Score (Macro): 0.4092

--- 라벨별 지표 ---
anger - Precision: 0.3906, Recall: 0.5569, F1-Score: 0.4591
disgust - Precision: 0.1230, Recall: 0.5882, F1-Score: 0.2034
fear - Precision: 0.2623, Recall: 0.6038, F1-Score: 0.3657
joy - Precision: 0.6621, Recall: 0.6097, F1-Score: 0.6348
sadness - Precision: 0.2964, Recall: 0.4740, F1-Score: 0.3647
surprise - Precision: 0.1938, Recall: 0.8532, F1-Score: 0.3159
neutral - Precision: 0.3893, Recall: 0.7855, F1-Score: 0.5206

Precision (Macro) 표준편차: 0.1624
Recall (Macro) 표준편차: 0.1231
F1-Score (Macro) 표준편차: 0.1313
